# Boka tid för dos 2-programmet
### Välkommen - följ instruktionerna nedan

1. I menyn "Run", välj "Run all cells" (längst ner i menyn).
2. Scrolla längst ner i det här dokumentet till rubriken "Använd programmet här"

<img src="https://m.media-amazon.com/images/I/719pUDFQy1L._SS500_.jpg" style="max-width:500px;" align="left">

In [ ]:
from pprint import pprint
import os
import re
import time
from time import sleep

from pynput.mouse import Button, Controller as MouseCtrlr
from pynput.keyboard import Key, Controller as KeyCtrlr
import pyautogui as pag
import keyboard as keyboard


from matplotlib import pyplot as plt
from PIL import Image
import cv2 as cv
import numpy as np

In [ ]:
# import pytesseract as tess
# tess.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

In [ ]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
from termcolor import colored, cprint

# Helper functions

In [ ]:
def imshow(img, cmap=None, size=10):
    if type(img) == str:
        img = cv.imread(img)
    elif(not type(img) == np.ndarray):
        return
    
    fig = plt.figure(figsize=(size,size))
    ax = plt.Axes(fig, [0,0,1,1])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(img, cmap)

In [ ]:
def get_inverted(image):
    return cv.bitwise_not(image)

# get grayscale image
def get_grayscale(image):
    return cv.cvtColor(image, cv.COLOR_BGR2GRAY)

#thresholding
def thresholding(image, threshold=180):
    return cv.threshold(image, threshold, 255, cv.THRESH_BINARY)[1]
    #return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)
    
def thresholdingAdaptive(image):
    return cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)

In [ ]:
kb = KeyCtrlr()

In [ ]:
class Pos:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"({self.x}, {self.y})"
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __bool__(self):
        return True

In [ ]:
def message(str):
    return pag.confirm(str)

In [ ]:
globalWaitTime = 1
globalLimitTime = 10
globalWaitBeforeStart = 0.2

def getWaitAndLimit(beforeStart = -1, limit = -1, wait = -1):
    if wait == -1:
        wait = globalWaitTime
    if limit == -1:
        limit = globalLimitTime
    if beforeStart == -1:
        beforeStart = globalWaitBeforeStart
        
    return (beforeStart, limit, wait)

In [ ]:
def waitForElement(path, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
        
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
            return Pos(x, y)
        except Exception as e:
            pass
            #print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element " + path)
    return False
        
#waitForElement("SokValjPatient")

In [ ]:
def waitForElementDisappear(path, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        try:
            x, y = pag.locateCenterOnScreen(path, confidence = conf)
        except Exception as e:
            if (str(e) == "cannot unpack non-iterable NoneType object"):
                return True
            else:
                print(str(e))
            
        sleep(waitTime)
        timePassed += waitTime
    
    if warn:
        message("Timed out waiting for element disappear " + path)
    return False


In [ ]:
def waitForOneOf(pathList, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True):
    timePassed = 0
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart, limit, waitTime)
    sleep(beforeStart)
    
    while timePassed < limit:
        for path in pathList:
            pos = waitForElement(path, beforeStart = 0, limit = 0.01, waitTime = 0.01, warn = False)
            timePassed += 0.01
            if pos:
                return pos
            
        sleep(waitTime)
        timePassed += waitTime
        
    if warn:
        message("Timed out waiting for element list " + str(pathList))
        
    return False
    
#waitForOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"], limit=2)

In [ ]:
def waitAndClickOneOf(pathList, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button = "left"):
    pos = waitForOneOf(pathList, beforeStart = beforeStart, limit = limit, waitTime = waitTime, conf = conf, warn = warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
            
        return pos
    
    return False


#waitAndClickOneOf(["FleraIFoljd", "osrsakskod1IcdFalt", "sokDiagnos"])

In [ ]:
def waitAndClickElement(path, relX = 0, relY = 0, beforeStart = -1, limit = -1, waitTime = -1, conf = 0.9, warn = True, button="left"):
    pos = waitForElement(path, beforeStart, limit, waitTime, conf, warn=warn)
    
    if pos:
        pos.x += relX
        pos.y += relY
        #print(f'Moving to {pos.x}, {pos.y}')
        
        if button == "left":
            pag.leftClick(pos.x, pos.y)
        elif button == "right":
            pag.rightClick(pos.x, pos.y)
        elif button == "double":
            pag.doubleClick(pos.x, pos.y)
        return pos
    else:
        return False

#waitAndClickElement("SokValjPatient")

In [ ]:
def waitAndClickXY(x, y, beforeStart = -1, waitTime = -1, button="left"):
    pos = Pos(x, y)
    return waitAndClick([pos], beforeStart = beforeStart, waitTime = waitTime, button=button)

def waitAndClick(positions, beforeStart = -1, waitTime = -1, button="left"):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    try:
        if type(positions) != list and type(positions) != tuple:
            positions = [positions]

        #print(f"Sleeping {beforeStart} before starting")
        sleep(beforeStart)
        for i, pos in enumerate(positions):
            if type(pos) == list or type(pos) == tuple:
                posObj = Pos(pos[0], pos[1])
            else:
                posObj = pos
            
            #print(text)
            if button == "left":
                pag.leftClick(posObj.x, posObj.y)
            elif button == "right":
                pag.rightClick(posObj.x, posObj.y)
            if i != len(positions) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndClick: " + str(e))
        return False
    

In [ ]:
def typeSlowly(text, keyInterval):
    for ch in text:
        kb.type(ch)
        sleep(keyInterval)


In [ ]:
def waitAndType(texts, beforeStart = -1, waitTime = -1, keyInterval=0):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart = beforeStart, wait = waitTime)
    
    if keyInterval == True:
        keyInterval = 0.05
    
    try:
        if type(texts) == str:
            texts = [texts]

        sleep(beforeStart)
        for i, text in enumerate(texts):
            #print(text)
            if keyInterval <= 0:
                kb.type(text)
            else:
                typeSlowly(text, keyInterval)
                
            if i != len(texts) - 1:
                sleep(waitTime)
        
        return True
    except Exception as e:
        print("Faild at waitAndType: " + str(e))
        return False
        
        
#waitAndType(["hej", "hej2", "hej3"])
#waitAndType("hejdå")

In [ ]:
def waitAndPress(key1, key2 = None, key3 = None, beforeStart = -1):
    beforeStart, limit, waitTime = getWaitAndLimit(beforeStart)
    
    sleep(beforeStart)
    
    try:
        if key2 and key3:
            keyboard.press_and_release(key1+"+"+key2+"+"+key3)
        elif key2:
            keyboard.press_and_release(key1+"+"+key2)
        else:
            keyboard.press_and_release(key1)
        return True
    except Exception as e:
        pag.confirm(f"Failed wait and press {str(key1)},{str(key2)},{str(key3)}")
        return False
    

In [ ]:
def AsError(text):
    return colored(text, "red")

def AsSuccess(text):
    return colored(text, "green")

def AsWarning(text):
    return colored(text, "yellow")

In [ ]:
def PrintSuccessStatus(result, title):
    outputText = title + ":"
    resultMessage = ""
    if result:
        resultMessage = AsSuccess("Lyckades!")
    else:
        resultMessage = AsError("MISSLYCKADES!")
    print(outputText, resultMessage)
    return result

# Data

In [ ]:
vaccination1 = {
    "name": "[Person som skrivit ut]",
    "rawData": """Tidbok Sida 1 av 1
VE Hälsocentralen Sankt Hans Lund
Fäladstorget 22
226 47
Tel
Fax
046-19 93 00
LUND
[Person som skrivit ut], ST-läkare
[Person som skrivit ut] Datum: Tidbok: ST-läkare
Tid Beskrivning
2021-05-13
08:00 -
<Ingen>
08:10
-
09:00
Test Häst, Hasse
19 501113-2098
<Ingen>
09:10
-
09:10
Test Kluck, Klara
19 501128-2182
<Ingen>
09:20
-
09:30
Test Klättermus, Klas
19 450630-4718
<Ingen>
09:40
-
10:10
Test Långstrump, Pippi
19 571230-2842
<Ingen>
10:20
-
2021-05-13 Utskriven av [Person som skrivit ut], ST-läkare
Tidbok Sida 2 av 2
VE Hälsocentralen Sankt Hans Lund
Fäladstorget 22
226 47
Tel
Fax
046-19 93 00
LUND
[Person som skrivit ut], ST-läkare
VE Hälsocentralen Sankt Hans Lund : Vaccinationsmottagning Datum: Tidbok: Tid Beskrivning
2021-05-15
09:00
Test Häst, Hasse
19 501113-2098
<Ingen>
09:10
-
09:10
Test Kluck, Klara
19 501128-2182
<Ingen>
09:20
-
09:30
Test Klättermus, Klas
19 450630-4718
<Ingen>
09:40
-
10:10
Test Långstrump, Pippi
19 571230-2842
<Ingen>
10:20
-
2021-05-13 Utskriven av [Person som skrivit ut], ST-läkare"""
}

In [ ]:
exampleStr = "\n10:10\nTest Långstrump, Pippi\n19 571230-2842\n<Ingen>\n10:20\n"

def getPersonalNoAndTime(rawPatientData):

    #print(f"Trying with\n{rawPatientData}\n")
    time = re.search(r"\d\d:\d\d", rawPatientData)[0]
    personalNo = re.search(r"\d{2} \d{6}-\d{4}", rawPatientData)
    
    if not personalNo:
        return None
    
    else:
        personalNo = personalNo[0]
        
        personalNo = re.sub(r" ", "", personalNo)

        patientData = {
            "time": time,
            "personalNo": personalNo
        }

        return patientData
    
    
#print(getPersonalNoAndTime(exampleStr))

In [ ]:
def filterScheduleData(rawData):
    relevantPart = re.search(r"Tid\s+Beskrivning\n\d\d\d\d-\d\d-\d\d\n([\w\W]+)-\n\d\d\d\d-\d\d-\d\d Utskriven.+", rawData)[1]
    relevantPart = re.sub(r"\d\d\d\d-\d\d-\d\d[\w\W]+?\d\d\d\d-\d\d-\d\d\n", "", relevantPart)
    dataList = relevantPart.split("\n-")
    
    patientList = []
    
    for rawPatientData in dataList:
        patientData = getPersonalNoAndTime(rawPatientData)
        if patientData:
            patientList.append(patientData)
        
    return patientList
    
pprint(filterScheduleData(vaccination1["rawData"]))

# Complex function

In [ ]:
class ImgsEnum:
    def __init__(self, imagesPath):
        self.ArbetsflodeLabel = os.path.join(imagesPath, "Vaccine", "ArbetsflodeLabel.png")
        self.StangButtonText = os.path.join(imagesPath, "Vaccine", "StangButtonText.png")
        self.NyDiagnosTab = os.path.join(imagesPath, "Vaccine", "NyDiagnosTab.png")
        self.KshDropdownChoice = os.path.join(imagesPath, "Vaccine", "KshDropdownChoice.png")
        self.Icd10DropdownChoice = os.path.join(imagesPath, "Vaccine", "icd10DropdownChoice.png")
        self.IcdSearchInputField = os.path.join(imagesPath, "Vaccine", "icdSearchInputField.png")
        self.NyVaccinationSearchField = os.path.join(imagesPath, "Vaccine", "NyVaccinationSearchField.png")
        self.NyAtgardTab = os.path.join(imagesPath, "Vaccine", "NyAtgardTab.png")
        self.Covid19AtgardChoice = os.path.join(imagesPath, "Vaccine", "covid19AtgardChoice.png")
        self.VaccinationCovid19Choice = os.path.join(imagesPath, "Vaccine", "VaccinationCovid19Choice.png")
        self.AtgardRelateradCovid19 = os.path.join(imagesPath, "Vaccine", "AtgardRelateradCovid19.png")
        self.CitrixIcon = os.path.join(imagesPath, "Vaccine", "CitrixIcon.png")
        self.CitrixIcon2 = os.path.join(imagesPath, "Vaccine", "CitrixIcon2.png")
        self.LotNrLabel = os.path.join(imagesPath, "Vaccine", "LotNrLabel.png")
        self.LotNrLabel2 = os.path.join(imagesPath, "Vaccine", "LotNrLabel2.png")
        self.DosNummerInputField = os.path.join(imagesPath, "Vaccine", "DosNummerInputField.png")
        self.KontraindikationNejCheckbox = os.path.join(imagesPath, "Vaccine", "KontraindikationNejCheckbox.png")
        self.NyVardkontaktTitle = os.path.join(imagesPath, "Vaccine", "NyVardkontaktTitle.png")
        self.Covid19VaccinationPatientCheckbox = os.path.join(imagesPath, "Vaccine", "Covid19VaccinationPatientCheckbox.png")
        self.VaccinationsmottagningCheckbox = os.path.join(imagesPath, "Vaccine", "VaccinationsmottagningCheckbox.png")
        self.NyDiagnosTitle = os.path.join(imagesPath, "Vaccine", "NyDiagnosTitle.png")
        self.NyAtgardTitle = os.path.join(imagesPath, "Vaccine", "NyAtgardTitle.png")
        self.NyVaccinationTab = os.path.join(imagesPath, "Vaccine", "NyVaccinationTab.png")
        self.SigneraDisabeledMenu = os.path.join(imagesPath, "Vaccine", "SigneraDisabeledMenu.png")
        self.NyBokningTitle = os.path.join(imagesPath, "Vaccine", "NyBokningTitle.png")
        self.LaggTillButton = os.path.join(imagesPath, "Vaccine", "LaggTillButton.png")
        self.OppnaJournalSokPatient = os.path.join(imagesPath, "Vaccine", "OppnaJournalSokPatient.png")
        self.Patientlista1TraffarLabel = os.path.join(imagesPath, "Vaccine", "Patientlista1TraffarLabel.png")
        self.DatumLabel = os.path.join(imagesPath, "Vaccine", "DatumLabel.png")
        self.WarningSymbolDoubleBooking = os.path.join(imagesPath, "Vaccine", "WarningSymbolDoubleBooking.png")
        self.AvbrytMenuButton = os.path.join(imagesPath, "Vaccine", "AvbrytMenuButton.png")
        self.WarningTrianglePopup = os.path.join(imagesPath, "Vaccine", "WarningTrianglePopup.png")
        self.TaBortButtonDisabled = os.path.join(imagesPath, "Vaccine", "TaBortButtonDisabled.png")
        self.DeltagareOchResurserTitle = os.path.join(imagesPath, "Vaccine", "DeltagareOchResurserTitle.png")
        self.DoljI1177Checkbox = os.path.join(imagesPath, "Vaccine", "DoljI1177Checkbox.png")
        #self. = os.path.join(imagesPath, "Vaccine", ".png")
        #self. = os.path.join(imagesPath, "Vaccine", ".png")

        
        #self. = os.path.join(imagesPath, "Vaccine", ".png")

    
IMG = ImgsEnum(os.path.join("..", "Images"))

IMG.ArbetsflodeLabel

In [ ]:
def schedulePatient(patientData, newDate, receptionName):
    waitAndPress("alt", "n")
    
    # Check for warning about trying to book on existing time:
    pos = waitForElement(IMG.NyBokningTitle)
    warningPos = waitForElement(IMG.WarningTrianglePopup, limit = 1, warn=False)
    if warningPos:
        waitAndPress("tab")
        waitAndPress("enter")

    pos = waitForElement(IMG.NyBokningTitle)
    
    if pos:
        sleep(0.2)
        waitAndPress("tab")
        waitAndPress("enter")
        pos = waitForElement(IMG.OppnaJournalSokPatient)
    if pos:
        waitAndType(patientData["personalNo"])
        waitAndPress("enter")
        pos = waitForElement(IMG.Patientlista1TraffarLabel)
    if pos:
        pos = waitAndClickXY(pos.x, pos.y + 45)
        waitAndPress("enter")
    if pos:
        sleep(0.2)
        pos = waitAndClickElement(IMG.DoljI1177Checkbox, relY = -33)
        
        pos = waitAndClickElement(IMG.TaBortButtonDisabled, relX = -80, conf=.99)
    if pos:
        pos = waitForElement(IMG.DeltagareOchResurserTitle)
    if pos:
        for i in range(4):
            waitAndPress("tab")
        sleep(0.1)
        typeSlowly(receptionName, 0.05)
        waitAndPress("enter")
        waitAndPress("alt", "o")
    if pos:
        sleep(0.2)
        pos = waitAndClickElement(IMG.DatumLabel, relY = 15)
    if pos:
        waitAndPress("ctrl", "a")
        waitAndType(newDate)
        waitAndPress("tab")
        waitAndType(patientData["time"])
        waitAndPress("tab")
        waitAndPress("tab")
        waitAndPress("tab")
        
        doubleBooking = waitForElement(IMG.WarningSymbolDoubleBooking, limit = 0.2, warn = False)
        
        if doubleBooking:
            pos = None
            
            print(AsWarning(f"<DUBBELBOKNING KL {patientData['time']} - BOKAR EJ!>"))
            cancelPos = waitAndClickElement(IMG.AvbrytMenuButton)
            if cancelPos:
                cancelPos = waitForElement(IMG.WarningTrianglePopup)
            if cancelPos:
                waitAndPress("enter")
                sleep(0.5)
                return cancelPos
            
        else:
            waitAndType("Dos 2")
            waitAndPress("alt", "s")
                
                
    return pos

In [ ]:
#pos = PrintSuccessStatus(waitAndClickOneOf([IMG.CitrixIcon, IMG.CitrixIcon2]), " - Öppnar PMO")
#schedulePatient(patientList[2], "2021-05-17", "Rasmus Arvidsson")

In [ ]:
def RunFullRebookingSequence(newDate, receptions):
    print(f"""Registrerar med...
Nytt datum: {newDate}
Mottagningar: {[x['name'] for x in receptions if len(x["data"]) > 0]}
""")
    
    finishMessage = ""
    
    if newDate == "":
        print(AsError("FEL - Du måste ange ett datum"))
        return
    
    finishMessage = "Misslyckades med att öppna PMO. Är det startat?"
    
    pos = PrintSuccessStatus(waitAndClickOneOf([IMG.CitrixIcon, IMG.CitrixIcon2]), " - Öppnar PMO")

    
    for reception in receptions:
        if len(reception["data"]) > 0:
            print(f"\n\nKÖR NU {reception['name']}\n")
            #pprint(reception["data"])
            patientList = reception["data"]

            if pos:
                for patient in patientList:
                    if pos:
                        pos = PrintSuccessStatus(schedulePatient(patient, newDate, reception["name"]), f" - Registrerar pat {newDate} kl {patient['time']}")
                        sleep(1)
 
        
    if pos:
        print(AsSuccess("\nAVSLUTAR"))
        finishMessage = "Redo för en ny omgång."
    else:
        print(AsError("\nAVSLUTAR"))
        
    print("\n - " + finishMessage)


# User interface

In [ ]:
def extractPatientDataFromTextareas(textareas):
    receptions = []
    anySuccess = False
    
    no = 1
    for textarea in textareas:
        currentReception = {"name": f"Vaccinationsmottagning {no}",
                            "rawData": textarea.value,
                            "data": []}
        
        try:
            patientList = filterScheduleData(currentReception["rawData"])
            currentReception["data"] = patientList
            print(f"- Reading data for reception {no}", AsSuccess(f"Succeeded!"))
            anySuccess = True
        except Exception as e:
            print(f"- Reading data for reception {no}", AsWarning(f"Could not read data!"))
        
        receptions.append(currentReception)
        no += 1
        
    return receptions, anySuccess


#tempRec = extractPatientDataFromTextareas(vaccReceptions)

In [ ]:
vaccReceptions = []

for i in range(1,7):
    newRec = widgets.Textarea(value='',
                              description=f'Vacc.mott. {i}:', )
    
    vaccReceptions.append(newRec)


dateTextField =  widgets.Text(
                  value='',
                  description='Skriv nytt datum (yyyy-mm-dd):', )

startButton = widgets.Button(description='Starta', 
                             style=widgets.ButtonStyle(button_color="rgb(40,80,200)"))

outputArea =  widgets.Output()


Interface = widgets.VBox(vaccReceptions + [dateTextField, 
                          startButton, 
                          outputArea])

def testOutput():
    print("Testing!")

def onButtonClicked(_):
    # "linking function with output"
    with outputArea:
        # what happens when we press the button
        clear_output()
        
        receptions, success = extractPatientDataFromTextareas(vaccReceptions)
        if not success:
            print (AsError("Lyckades inte läsa någon data. AVSLUTAR"))
        
        RunFullRebookingSequence(dateTextField.value, receptions)
        
# linking button and function together using a button's method
startButton.on_click(onButtonClicked)



<br/>
<br/>
<img src="https://media.istockphoto.com/vectors/cute-cartoon-car-set-with-dash-line-and-horizontal-road-baby-kids-vector-id1141945630" style="max-width:500px;display:block;" align="left">

# Använd programmet här!
1. Välj vaccin, dos och skriv in batch nummer.
2. Dubbelklicka på rätt patient i tidboken i PMO, men gå inte in i journalen.
3. Klicka på Starta.
4. När du är klar med en patient kan du börja om från nr 2 ovan med nästa patient.

In [ ]:
Interface

<img src="https://cdn.pixabay.com/photo/2014/10/19/04/52/light-494011__340.png" style="max-width:500px;display:block;" align="left">
